In [2]:
import csv
import requests
from bs4 import BeautifulSoup
import os.path
import pandas as pd
from random import uniform
from time import sleep



def get_soup(url):
    try:
        sleep(uniform(.3, 3.5))
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        get_soup(url)
        
    return soup


In [12]:
import pandas as pd
restos = pd.read_csv('to_add.csv', encoding = 'Latin1')
restos = restos.sort_values(by = 'restos', ascending = True)
restos = restos.restos.values
restos[14]

'cho-dang-gol'

In [ ]:
from random import uniform
from time import sleep
import os
import codecs

areas = ['new-york', 'brooklyn', 'long-island-city', 'new-york-4']
error_log = 'error.csv'
yelp_data_file = 'yelp_data_additional.csv'

for resto in restos[14:]:
    area      = areas[0]
    home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format(resto, areas[0], '0')
    soup = get_soup(home_page)
    
    try:
        n_pages = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip()) 
    except:
        try:
            home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format(resto, areas[1], '0')
            soup      = get_soup(home_page)
            n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip()) 
            area      = areas[1]
        except:
            try:
                home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format(resto, areas[2], '0')
                soup      = get_soup(home_page)
                n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip()) 
                area       = areas[2]
            except:
                try:
                    home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format('the-'+ resto, areas[0], '0')
                    soup      = get_soup(home_page)
                    n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip())
                    area      = areas[0]
                    resto = 'the-' + resto
                except:
                    try:
                        home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format('the-'+ resto, areas[1], '0')
                        soup      = get_soup(home_page)
                        n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip())
                        area      = areas[1]
                        resto = 'the-' + resto
                    except:
                        try:
                            home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format('the-'+ resto, areas[2], '0')
                            soup      = get_soup(home_page)
                            n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip())
                            area      = areas[2]
                            resto = 'the-' + resto
                        except:
                            try:
                                home_page = 'https://www.yelp.com/biz/{}-{}?start={}'.format('the-'+ resto, areas[3], '0')
                                soup      = get_soup(home_page)
                                n_pages   = int(soup.find_all(class_ = 'page-of-pages')[0].text.strip('\n').split('of')[1].strip())
                                area      = areas[3]
                                resto = 'the-' + resto
                            except:
                                with open(error_log, 'a') as csv_file:
                                    writer = csv.writer(csv_file)
                                    writer.writerow([resto])
                                    n_pages = 0
                            

    for p in range(0, n_pages + 1):
        
        url     = 'https://www.yelp.com/biz/{}-{}?start={}&sort_by=date_asc'.format(resto, area, p*20)
        url_id  = 'scrape/{}_{}.html'.format(resto, p*20)
        
        if os.path.isfile(url_id) is True:
            f    = codecs.open(url_id, 'r')
            a    = f.read()
            soup =  BeautifulSoup(a, 'html.parser')
        else:
            soup = get_soup(url)
            
            with open(url_id, "w") as file:
                file.write(str(soup))
        
        review_boxes = soup.find_all(class_='review-content')
        
        
        for r in range(0,len(review_boxes)):
            row = list()
            row.append(resto)
            row.append((review_boxes[r].find(class_ = 'rating-qualifier')
                       .text.strip()
                       .rstrip()
                       .replace('\n', '')
                       .replace('Updated review', '').
                       split(' ')[0] ))
            row.append( float(review_boxes[r].find(class_ = 'i-stars').img['alt'].split(' ')[0] ))
            row.append(review_boxes[r].find('p').text) 
            with open(yelp_data_file, "a") as csv_file:
                writer = csv.writer(csv_file)
                writer.writerow(row)
            
        

                
